In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 74 kB/s 


In [ ]:
class config:
  run_type='colab'
  random_seed=2022
  nfold=5
  target = 'pm25_mid'

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

%matplotlib inline

from matplotlib import pyplot as plt
import seaborn as sns
import re

import sklearn.preprocessing as preproc

import json, os, gc, math, time
import datetime
import collections
from tqdm import tqdm
import glob

from statistics import mean
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

from sklearn import metrics
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#ドライブへのアクセス
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/00_datascience/09_sony"

Mounted at /content/drive
/content/drive/My Drive/00_datascience/09_sony


In [ ]:
#データの取得
train = pd.read_csv("/content/drive/MyDrive/00_datascience/09_sony/input/train.csv")
test = pd.read_csv("/content/drive/MyDrive/00_datascience/09_sony/input/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/00_datascience/09_sony/input/submit_sample.csv", header=None)
external_data = pd.read_csv('/content/drive/MyDrive/00_datascience/09_sony/input/external_data_small.csv')
target = train[config.target]
train_for_split = train.copy()

# 外部データとの連結

In [ ]:
#外部データとの結合
train = pd.merge(train, external_data, how="left", on = "Country")

test = pd.merge(test, external_data, how="left", on = "Country")

In [ ]:
train.head()

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,bus,bike,car_per_1000people
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,2.590,2.633,0.850,29,2.397,4.295,5.916,3.017,11,0.149,0.468,0.900,0.137,36,0.000,0.529,2.188,1.167,36,10.956,14.038,20.866,23.880,45,30.207,70.426,94.202,994.051,59,1004.137,1015.721,1023.398,89.588,42,0.241,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,97.0,849,729
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,3.181,4.828,1.208,49,3.092,7.572,12.332,20.524,31,0.096,0.102,16.152,61.550,45,0.099,0.301,5.411,6.401,71,24.113,30.125,36.678,37.327,50,49.034,74.276,92.975,467.112,69,938.085,970.252,1080.000,7.320,48,0.828,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,97.0,849,729
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,1.197,2.200,0.248,123,0.736,12.527,23.432,125.224,36,1.392,1.464,47.652,515.570,111,0.972,2.751,12.230,30.518,148,0.000,0.000,23.226,131.280,138,25.919,69.069,100.000,1292.191,153,880.000,880.000,1018.200,4655.348,125,0.000,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,97.0,849,729
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,11.044,14.802,24.186,90,1.917,14.141,38.374,304.731,50,2.957,3.230,14.056,25.630,176,14.959,17.471,24.221,20.883,101,16.423,19.819,20.044,3.188,111,73.504,86.920,100.000,85.028,141,1011.703,1016.545,1018.364,10.898,52,0.284,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370,97.0,849,729
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,8.681,11.100,10.011,83,0.349,2.349,38.600,353.549,47,0.422,5.794,9.972,9.234,80,14.911,21.140,32.584,74.298,55,10.297,11.822,13.377,2.311,155,51.918,76.736,96.533,484.188,124,1019.521,1022.992,1027.611,15.142,102,0.500,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063,97.0,849,729


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195941 entries, 0 to 195940
Data columns (total 57 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  195941 non-null  int64  
 1   year                195941 non-null  int64  
 2   month               195941 non-null  int64  
 3   day                 195941 non-null  int64  
 4   Country             195941 non-null  object 
 5   City                195941 non-null  object 
 6   lat                 195941 non-null  float64
 7   lon                 195941 non-null  float64
 8   co_cnt              195941 non-null  int64  
 9   co_min              195941 non-null  float64
 10  co_mid              195941 non-null  float64
 11  co_max              195941 non-null  float64
 12  co_var              195941 non-null  float64
 13  o3_cnt              195941 non-null  int64  
 14  o3_min              195941 non-null  float64
 15  o3_mid              195941 non-nul

# weekごとに最もpm2.5が大きい / 小さいCityからの距離をとる

In [ ]:
train['week'] = pd.qcut(train['day'], 4, labels=False) + 1
test['week'] = pd.qcut(test['day'], 4, labels=False) + 1

train['week'].unique()

array([1, 2, 3, 4])

In [ ]:
from math import radians, cos, sin, asin, sqrt

def add_monthly_highest_pm25_city(train, test):
    monthly_pm25 =  train.groupby(['year', 'month', 'City'])[config.target].mean().to_frame(name='monthly_pm25').reset_index(drop=False)
    monthly_max_pm25 = monthly_pm25.groupby(['year', 'month'])['monthly_pm25'].max().to_frame(name='monthly_max_pm25').reset_index(drop=False)
    pm25 = monthly_pm25.merge(monthly_max_pm25, on=['year', 'month'], how='left')
    mothly_highest_city =  pm25[pm25['monthly_pm25'] == pm25['monthly_max_pm25']].drop(['monthly_pm25', 'monthly_max_pm25'], axis=1).rename({'City': 'monthly_highest_city'}, axis=1).reset_index(drop=True)
    train = train.merge(mothly_highest_city, how='left', on=['year', 'month'])
    test = test.merge(mothly_highest_city, how='left', on=['year', 'month'])
    return train, test

def add_dayly_highest_pm25_city(train, test):
    dayly_pm25 =  train.groupby(['year', 'month', 'day', 'City'])[config.target].mean().to_frame(name='dayly_pm25').reset_index(drop=False)
    dayly_max_pm25 = dayly_pm25.groupby(['year', 'month', 'day'])['dayly_pm25'].max().to_frame(name='dayly_max_pm25').reset_index(drop=False)
    pm25 = dayly_pm25.merge(dayly_max_pm25, on=['year', 'month', 'day'], how='left')
    dayly_highest_city =  pm25[pm25['dayly_pm25'] == pm25['dayly_max_pm25']].drop(['dayly_pm25', 'dayly_max_pm25'], axis=1).rename({'City': 'dayly_highest_city'}, axis=1).reset_index(drop=True)
    train = train.merge(dayly_highest_city, how='left', on=['year', 'month', 'day'])
    test = test.merge(dayly_highest_city, how='left', on=['year', 'month', 'day'])
    return train, test

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

def manhattan(lon1, lat1, lon2, lat2):
    return np.abs(lat2 - lat1) + np.abs(lon2 - lon1)

def euclidean(lon1, lat1, lon2, lat2):
    return ((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2) ** 0.5

def add_dist_from_monthly_highest_pm25(train, test):
    train, test = add_monthly_highest_pm25_city(train, test)
    city_loc = {}
    for city in train['monthly_highest_city'].unique():
        geographic = train[train['City'] == city].loc[:, ['lat', 'lon']].drop_duplicates().values[0]
        city_loc[city] = {}
        city_loc[city]['lat'] = geographic[0]
        city_loc[city]['lon'] = geographic[1]
    
    # ------ haversine  ------
    train['haversine_monthly_highest'] = train.apply(lambda x: haversine(x['lon'], x['lat'], city_loc[x['monthly_highest_city']]['lon'], city_loc[x['monthly_highest_city']]['lat']), axis=1)
    test['haversine_monthly_highest'] = test.apply(lambda x: haversine(x['lon'], x['lat'], city_loc[x['monthly_highest_city']]['lon'], city_loc[x['monthly_highest_city']]['lat']), axis=1)
    
    # ------ manhattan ------
    train['manhattan_monthly_highest'] = train.apply(lambda x: manhattan(x['lon'], x['lat'], city_loc[x['monthly_highest_city']]['lon'], city_loc[x['monthly_highest_city']]['lat']), axis=1)
    test['manhattan_monthly_highest'] = test.apply(lambda x: manhattan(x['lon'], x['lat'], city_loc[x['monthly_highest_city']]['lon'], city_loc[x['monthly_highest_city']]['lat']), axis=1)
    
    # ------ euclidean ------
    train['euclidean_monthly_highest'] = train.apply(lambda x: euclidean(x['lon'], x['lat'], city_loc[x['monthly_highest_city']]['lon'], city_loc[x['monthly_highest_city']]['lat']), axis=1)
    test['euclidean_monthly_highest'] = test.apply(lambda x: euclidean(x['lon'], x['lat'], city_loc[x['monthly_highest_city']]['lon'], city_loc[x['monthly_highest_city']]['lat']), axis=1)
    
    # ------ diff ------
    train['londiff_monthly_highest'] = train.apply(lambda x: x['lon'] - city_loc[x['monthly_highest_city']]['lon'], axis=1)
    train['latdiff_monthly_highest'] = train.apply(lambda x: x['lat'] - city_loc[x['monthly_highest_city']]['lat'], axis=1)
    test['londiff_monthly_highest'] = test.apply(lambda x: x['lon'] - city_loc[x['monthly_highest_city']]['lon'], axis=1)
    test['latdiff_monthly_highest'] = test.apply(lambda x: x['lat'] - city_loc[x['monthly_highest_city']]['lat'], axis=1)

    train.drop('monthly_highest_city', axis=1, inplace=True)
    test.drop('monthly_highest_city', axis=1, inplace=True)
    return train, test

def add_dist_from_dayly_highest_pm25(train, test):
    train, test = add_dayly_highest_pm25_city(train, test)
    city_loc = {}
    for city in train['dayly_highest_city'].unique():
        geographic = train[train['City'] == city].loc[:, ['lat', 'lon']].drop_duplicates().values[0]
        city_loc[city] = {}
        city_loc[city]['lat'] = geographic[0]
        city_loc[city]['lon'] = geographic[1]
    
    # ------ haversine  ------
    train['haversine_dayly_highest'] = train.apply(lambda x: haversine(x['lon'], x['lat'], city_loc[x['dayly_highest_city']]['lon'], city_loc[x['dayly_highest_city']]['lat']), axis=1)
    test['haversine_dayly_highest'] = test.apply(lambda x: haversine(x['lon'], x['lat'], city_loc[x['dayly_highest_city']]['lon'], city_loc[x['dayly_highest_city']]['lat']), axis=1)
    
    # ------ manhattan ------
    train['manhattan_dayly_highest'] = train.apply(lambda x: manhattan(x['lon'], x['lat'], city_loc[x['dayly_highest_city']]['lon'], city_loc[x['dayly_highest_city']]['lat']), axis=1)
    test['manhattan_dayly_highest'] = test.apply(lambda x: manhattan(x['lon'], x['lat'], city_loc[x['dayly_highest_city']]['lon'], city_loc[x['dayly_highest_city']]['lat']), axis=1)
    
    # ------ euclidean ------
    train['euclidean_dayly_highest'] = train.apply(lambda x: euclidean(x['lon'], x['lat'], city_loc[x['dayly_highest_city']]['lon'], city_loc[x['dayly_highest_city']]['lat']), axis=1)
    test['euclidean_dayly_highest'] = test.apply(lambda x: euclidean(x['lon'], x['lat'], city_loc[x['dayly_highest_city']]['lon'], city_loc[x['dayly_highest_city']]['lat']), axis=1)
    
    # ------ diff ------
    train['londiff_dayly_highest'] = train.apply(lambda x: x['lon'] - city_loc[x['dayly_highest_city']]['lon'], axis=1)
    train['latdiff_dayly_highest'] = train.apply(lambda x: x['lat'] - city_loc[x['dayly_highest_city']]['lat'], axis=1)
    test['londiff_dayly_highest'] = test.apply(lambda x: x['lon'] - city_loc[x['dayly_highest_city']]['lon'], axis=1)
    test['latdiff_dayly_highest'] = test.apply(lambda x: x['lat'] - city_loc[x['dayly_highest_city']]['lat'], axis=1)
    
    train.drop('dayly_highest_city', axis=1, inplace=True)
    test.drop('dayly_highest_city', axis=1, inplace=True)
    return train, test

In [ ]:
dayly_pm25 =  train.groupby(['year', 'month', 'day', 'City'])[config.target].mean().to_frame(name='dayly_pm25').reset_index(drop=False)
dayly_min_pm25 = dayly_pm25.groupby(['year', 'month', 'day'])['dayly_pm25'].min().to_frame(name='dayly_min_pm25').reset_index(drop=False)
pm25 = dayly_pm25.merge(dayly_min_pm25, on=['year', 'month', 'day'], how='left')
dayly_lowest_city =  pm25[pm25['dayly_pm25'] == pm25['dayly_min_pm25']].drop(['dayly_pm25', 'dayly_min_pm25'], axis=1).rename({'City': 'dayly_lowest_city'}, axis=1).reset_index(drop=True)

In [ ]:
def add_monthly_lowest_pm25_city(train, test):
    monthly_pm25 =  train.groupby(['year', 'month', 'City'])[config.target].mean().to_frame(name='monthly_pm25').reset_index(drop=False)
    monthly_min_pm25 = monthly_pm25.groupby(['year', 'month'])['monthly_pm25'].min().to_frame(name='monthly_min_pm25').reset_index(drop=False)
    pm25 = monthly_pm25.merge(monthly_min_pm25, on=['year', 'month'], how='left')
    mothly_lowest_city =  pm25[pm25['monthly_pm25'] == pm25['monthly_min_pm25']].drop(['monthly_pm25', 'monthly_min_pm25'], axis=1).rename({'City': 'monthly_lowest_city'}, axis=1).reset_index(drop=True)
    train = train.merge(mothly_lowest_city, how='left', on=['year', 'month'])
    test = test.merge(mothly_lowest_city, how='left', on=['year', 'month'])
    return train, test

def add_dayly_lowest_pm25_city(train, test):
    dayly_pm25 =  train.groupby(['year', 'month', 'day', 'City'])[config.target].mean().to_frame(name='dayly_pm25').reset_index(drop=False)
    dayly_min_pm25 = dayly_pm25.groupby(['year', 'month', 'day'])['dayly_pm25'].min().to_frame(name='dayly_min_pm25').reset_index(drop=False)
    pm25 = dayly_pm25.merge(dayly_min_pm25, on=['year', 'month', 'day'], how='left')
    dayly_lowest_city =  pm25[pm25['dayly_pm25'] == pm25['dayly_min_pm25']].drop(['dayly_pm25', 'dayly_min_pm25'], axis=1).rename({'City': 'dayly_lowest_city'}, axis=1).reset_index(drop=True)
    dayly_lowest_city.drop_duplicates(subset=['year', 'month', 'day'], keep='last', inplace=True)
    train = train.merge(dayly_lowest_city, how='left', on=['year', 'month', 'day'])
    test = test.merge(dayly_lowest_city, how='left', on=['year', 'month', 'day'])
    return train, test

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

def manhattan(lon1, lat1, lon2, lat2):
    return np.abs(lat2 - lat1) + np.abs(lon2 - lon1)

def euclidean(lon1, lat1, lon2, lat2):
    return ((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2) ** 0.5

def add_dist_from_monthly_lowest_pm25(train, test):
    train, test = add_monthly_lowest_pm25_city(train, test)
    city_loc = {}
    for city in train['monthly_lowest_city'].unique():
        geographic = train[train['City'] == city].loc[:, ['lat', 'lon']].drop_duplicates().values[0]
        city_loc[city] = {}
        city_loc[city]['lat'] = geographic[0]
        city_loc[city]['lon'] = geographic[1]
    
    # ------ haversine  ------
    train['haversine_monthly_lowest'] = train.apply(lambda x: haversine(x['lon'], x['lat'], city_loc[x['monthly_lowest_city']]['lon'], city_loc[x['monthly_lowest_city']]['lat']), axis=1)
    test['haversine_monthly_lowest'] = test.apply(lambda x: haversine(x['lon'], x['lat'], city_loc[x['monthly_lowest_city']]['lon'], city_loc[x['monthly_lowest_city']]['lat']), axis=1)
    
    # ------ manhattan ------
    train['manhattan_monthly_lowest'] = train.apply(lambda x: manhattan(x['lon'], x['lat'], city_loc[x['monthly_lowest_city']]['lon'], city_loc[x['monthly_lowest_city']]['lat']), axis=1)
    test['manhattan_monthly_lowest'] = test.apply(lambda x: manhattan(x['lon'], x['lat'], city_loc[x['monthly_lowest_city']]['lon'], city_loc[x['monthly_lowest_city']]['lat']), axis=1)
    
    # ------ euclidean ------
    train['euclidean_monthly_lowest'] = train.apply(lambda x: euclidean(x['lon'], x['lat'], city_loc[x['monthly_lowest_city']]['lon'], city_loc[x['monthly_lowest_city']]['lat']), axis=1)
    test['euclidean_monthly_lowest'] = test.apply(lambda x: euclidean(x['lon'], x['lat'], city_loc[x['monthly_lowest_city']]['lon'], city_loc[x['monthly_lowest_city']]['lat']), axis=1)
    
    # ------ diff ------
    train['londiff_monthly_lowest'] = train.apply(lambda x: x['lon'] - city_loc[x['monthly_lowest_city']]['lon'], axis=1)
    train['latdiff_monthly_lowest'] = train.apply(lambda x: x['lat'] - city_loc[x['monthly_lowest_city']]['lat'], axis=1)
    test['londiff_monthly_lowest'] = test.apply(lambda x: x['lon'] - city_loc[x['monthly_lowest_city']]['lon'], axis=1)
    test['latdiff_monthly_lowest'] = test.apply(lambda x: x['lat'] - city_loc[x['monthly_lowest_city']]['lat'], axis=1)

    train.drop('monthly_lowest_city', axis=1, inplace=True)
    test.drop('monthly_lowest_city', axis=1, inplace=True)
    return train, test

def add_dist_from_dayly_lowest_pm25(train, test):
    train, test = add_dayly_lowest_pm25_city(train, test)
    city_loc = {}
    for city in train['dayly_lowest_city'].unique():
        geographic = train[train['City'] == city].loc[:, ['lat', 'lon']].drop_duplicates().values[0]
        city_loc[city] = {}
        city_loc[city]['lat'] = geographic[0]
        city_loc[city]['lon'] = geographic[1]
    
    # ------ haversine  ------
    train['haversine_dayly_lowest'] = train.apply(lambda x: haversine(x['lon'], x['lat'], city_loc[x['dayly_lowest_city']]['lon'], city_loc[x['dayly_lowest_city']]['lat']), axis=1)
    test['haversine_dayly_lowest'] = test.apply(lambda x: haversine(x['lon'], x['lat'], city_loc[x['dayly_lowest_city']]['lon'], city_loc[x['dayly_lowest_city']]['lat']), axis=1)
    
    # ------ manhattan ------
    train['manhattan_dayly_lowest'] = train.apply(lambda x: manhattan(x['lon'], x['lat'], city_loc[x['dayly_lowest_city']]['lon'], city_loc[x['dayly_lowest_city']]['lat']), axis=1)
    test['manhattan_dayly_lowest'] = test.apply(lambda x: manhattan(x['lon'], x['lat'], city_loc[x['dayly_lowest_city']]['lon'], city_loc[x['dayly_lowest_city']]['lat']), axis=1)
    
    # ------ euclidean ------
    train['euclidean_dayly_lowest'] = train.apply(lambda x: euclidean(x['lon'], x['lat'], city_loc[x['dayly_lowest_city']]['lon'], city_loc[x['dayly_lowest_city']]['lat']), axis=1)
    test['euclidean_dayly_lowest'] = test.apply(lambda x: euclidean(x['lon'], x['lat'], city_loc[x['dayly_lowest_city']]['lon'], city_loc[x['dayly_lowest_city']]['lat']), axis=1)
    
    # ------ diff ------
    train['londiff_dayly_lowest'] = train.apply(lambda x: x['lon'] - city_loc[x['dayly_lowest_city']]['lon'], axis=1)
    train['latdiff_dayly_lowest'] = train.apply(lambda x: x['lat'] - city_loc[x['dayly_lowest_city']]['lat'], axis=1)
    test['londiff_dayly_lowest'] = test.apply(lambda x: x['lon'] - city_loc[x['dayly_lowest_city']]['lon'], axis=1)
    test['latdiff_dayly_lowest'] = test.apply(lambda x: x['lat'] - city_loc[x['dayly_lowest_city']]['lat'], axis=1)
    
    train.drop('dayly_lowest_city', axis=1, inplace=True)
    test.drop('dayly_lowest_city', axis=1, inplace=True)
    return train, test

In [ ]:
#最大のcityからの距離
train, test = add_dist_from_dayly_highest_pm25(train, test)
train, test = add_dist_from_monthly_highest_pm25(train, test)

In [ ]:
#最小のcityからの距離
train, test = add_dist_from_dayly_lowest_pm25(train, test)
train, test = add_dist_from_monthly_lowest_pm25(train, test)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195941 entries, 0 to 195940
Data columns (total 78 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         195941 non-null  int64  
 1   year                       195941 non-null  int64  
 2   month                      195941 non-null  int64  
 3   day                        195941 non-null  int64  
 4   Country                    195941 non-null  object 
 5   City                       195941 non-null  object 
 6   lat                        195941 non-null  float64
 7   lon                        195941 non-null  float64
 8   co_cnt                     195941 non-null  int64  
 9   co_min                     195941 non-null  float64
 10  co_mid                     195941 non-null  float64
 11  co_max                     195941 non-null  float64
 12  co_var                     195941 non-null  float64
 13  o3_cnt                     19

# 時系列に関わるデータの処理

In [ ]:
#月の順番を変えたデータの付与
month_convert = pd.DataFrame({
        'month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'month_converted': [12, 11, 6, 7, 4, 3, 1, 2, 5, 8, 9, 10]
})

In [ ]:
#最もpm25が多い月、少ない月からの距離
month_distance = pd.DataFrame({
        'month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'month_distance': [0, 1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1]
})

In [ ]:
#yyyymmdd
train['yyyymmdd'] = train['year'] * 10000 + train['month'] * 100 + train['day']
test['yyyymmdd'] = test['year'] * 10000 + test['month'] * 100 + test['day']

In [ ]:
#作成した月データとのマージ
train = pd.merge(train, month_convert, how="left", on = "month")
train = pd.merge(train, month_distance, how="left", on = "month")

test = pd.merge(test, month_convert, how="left", on = "month")
test = pd.merge(test, month_distance, how="left", on = "month")

In [ ]:
train.head()

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,bus,bike,car_per_1000people,week,haversine_dayly_highest,manhattan_dayly_highest,euclidean_dayly_highest,londiff_dayly_highest,latdiff_dayly_highest,haversine_monthly_highest,manhattan_monthly_highest,euclidean_monthly_highest,londiff_monthly_highest,latdiff_monthly_highest,haversine_dayly_lowest,manhattan_dayly_lowest,euclidean_dayly_lowest,londiff_dayly_lowest,latdiff_dayly_lowest,haversine_monthly_lowest,manhattan_monthly_lowest,euclidean_monthly_lowest,londiff_monthly_lowest,latdiff_monthly_lowest,yyyymmdd,month_converted,month_distance
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,2.590,2.633,0.850,29,2.397,4.295,5.916,3.017,11,0.149,0.468,0.900,0.137,36,0.000,0.529,2.188,1.167,36,10.956,14.038,20.866,23.880,45,30.207,70.426,94.202,994.051,59,1004.137,1015.721,1023.398,89.588,42,0.241,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,97.0,849,729,1,10199.456268,131.91649,94.311010,75.79660,-56.11989,10199.456268,131.91649,94.311010,75.79660,-56.11989,7328.072022,80.47936,66.136493,16.41142,-64.06794,13010.294810,168.67617,122.643378,104.53181,-64.14436,20190101,12,0
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,3.181,4.828,1.208,49,3.092,7.572,12.332,20.524,31,0.096,0.102,16.152,61.550,45,0.099,0.301,5.411,6.401,71,24.113,30.125,36.678,37.327,50,49.034,74.276,92.975,467.112,69,938.085,970.252,1080.000,7.320,48,0.828,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,97.0,849,729,1,7351.674654,94.72344,67.560018,53.61036,-41.11308,7351.674654,94.72344,67.560018,53.61036,-41.11308,5488.824963,54.83595,49.399828,-5.77482,-49.06113,10164.105029,131.48312,95.892084,82.34557,-49.13755,20190101,12,0
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,1.197,2.200,0.248,123,0.736,12.527,23.432,125.224,36,1.392,1.464,47.652,515.570,111,0.972,2.751,12.230,30.518,148,0.000,0.000,23.226,131.280,138,25.919,69.069,100.000,1292.191,153,880.000,880.000,1018.200,4655.348,125,0.000,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,97.0,849,729,1,10206.768673,134.19778,94.896382,67.73183,-66.46595,10206.768673,134.19778,94.896382,67.73183,-66.46595,8318.849546,82.76065,74.880638,8.34665,-74.41400,12892.900183,170.95746,121.879910,96.46704,-74.49042,20190101,12,0
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,11.044,14.802,24.186,90,1.917,14.141,38.374,304.731,50,2.957,3.230,14.056,25.630,176,14.959,17.471,24.221,20.883,101,16.423,19.819,20.044,3.188,111,73.504,86.920,100.000,85.028,141,1011.703,1016.545,1018.364,10.898,52,0.284,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370,97.0,849,729,1,10418.232918,136.13009,96.694229,74.54861,-61.58148,10418.232918,136.13009,96.694229,74.54861,-61.58148,7890.150242,84.69296,71.163791,15.16343,-69.52953,13192.648423,172.88977,124.549331,103.28382,-69.60595,20190101,12,0
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,8.681,11.100,10.011,83,0.349,2.349,38.600,353.549,47,0.422,5.794,9.972,9.234,80,14.911,21.140,32.584,74.298,55,10.297,11.822,13.377,2.311,155,51.918,76.736,96.533,484.188,124,1019.521,1022.992,1027.611,15.142,102,0.500,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063,97.0,849,729,1,7878.988579,99.23410,71.868893,38.62991,-60.60419,7878.988579,99.23410,71.868893,38.62991,-60.60419,7922.595805,89.30751,71.625351,-20.75527,-68.55224,10353.007068,135.99378,96.166275,67.36512,-68.62866,20190101,12,0


# Countryをラベルエンコーディング

In [ ]:
#ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

cat_cols = ['Country']

#カテゴリ変数をループしてlabel encoding
for c in cat_cols:
  le = LabelEncoder()
  le.fit(train[c])
  train[c] = le.transform(train[c])
  test[c] = le.transform(test[c])

train[cat_cols] = train[cat_cols]+1
test[cat_cols] = test[cat_cols]+1

# 組み合わせによる特徴量の作成

In [ ]:
#集約特徴量 train
train['co_o3_mid'] = train['co_mid'] + train['o3_mid']
train['co_so2_mid'] = train['co_mid'] + train['so2_mid']
train['co_no2_mid'] = train['co_mid'] + train['no2_mid']
train['o3_so2_mid'] = train['o3_mid'] + train['so2_mid']
train['o3_no2_mid'] = train['o3_mid'] + train['no2_mid']
train['so2_no2_mid'] = train['so2_mid'] + train['no2_mid']

In [ ]:
#集約特徴量 test
test['co_o3_mid'] = test['co_mid'] + test['o3_mid']
test['co_so2_mid'] = test['co_mid'] + test['so2_mid']
test['co_no2_mid'] = test['co_mid'] + test['no2_mid']
test['o3_so2_mid'] = test['o3_mid'] + test['so2_mid']
test['o3_no2_mid'] = test['o3_mid'] + test['no2_mid']
test['so2_no2_mid'] = test['so2_mid'] + test['no2_mid']

In [ ]:
train['tmp_hum_mid'] = train['temperature_mid'] * train['humidity_mid']
train['tmp_pre_mid'] = train['temperature_mid'] * train['pressure_mid']
train['tmp_ws_mid'] = train['temperature_mid'] * train['ws_mid']
train['tmp_dew_mid'] = train['temperature_mid'] * train['dew_mid']
train['hum_pre_mid'] = train['humidity_mid'] * train['pressure_mid']
train['hum_ws_mid'] = train['humidity_mid'] * train['ws_mid']
train['hum_dew_mid'] = train['humidity_mid'] * train['dew_mid']
train['pre_ws_mid'] = train['pressure_mid'] * train['ws_mid']
train['pre_dew_mid'] = train['pressure_mid'] * train['dew_mid']
train['ws_dew_mid'] = train['ws_mid'] * train['dew_mid']

In [ ]:
test['tmp_hum_mid'] = test['temperature_mid'] * test['humidity_mid']
test['tmp_pre_mid'] = test['temperature_mid'] * test['pressure_mid']
test['tmp_ws_mid'] = test['temperature_mid'] * test['ws_mid']
test['tmp_dew_mid'] = test['temperature_mid'] * test['dew_mid']
test['hum_pre_mid'] = test['humidity_mid'] * test['pressure_mid']
test['hum_ws_mid'] = test['humidity_mid'] * test['ws_mid']
test['hum_dew_mid'] = test['humidity_mid'] * test['dew_mid']
test['pre_ws_mid'] = test['pressure_mid'] * test['ws_mid']
test['pre_dew_mid'] = test['pressure_mid'] * test['dew_mid']
test['ws_dew_mid'] = test['ws_mid'] * test['dew_mid']

In [ ]:
#train = pd.merge(train, pair_train, left_index=True, right_index=True)
#test = pd.merge(test, pair_test, left_index=True, right_index=True)

In [ ]:
train.head()

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,bus,bike,car_per_1000people,week,haversine_dayly_highest,manhattan_dayly_highest,euclidean_dayly_highest,londiff_dayly_highest,latdiff_dayly_highest,haversine_monthly_highest,manhattan_monthly_highest,euclidean_monthly_highest,londiff_monthly_highest,latdiff_monthly_highest,haversine_dayly_lowest,manhattan_dayly_lowest,euclidean_dayly_lowest,londiff_dayly_lowest,latdiff_dayly_lowest,haversine_monthly_lowest,manhattan_monthly_lowest,euclidean_monthly_lowest,londiff_monthly_lowest,latdiff_monthly_lowest,yyyymmdd,month_converted,month_distance,co_o3_mid,co_so2_mid,co_no2_mid,o3_so2_mid,o3_no2_mid,so2_no2_mid,tmp_hum_mid,tmp_pre_mid,tmp_ws_mid,tmp_dew_mid,hum_pre_mid,hum_ws_mid,hum_dew_mid,pre_ws_mid,pre_dew_mid,ws_dew_mid
0,1,2019,1,1,1,Brisbane,-27.46794,153.02809,38,0.749,2.590,2.633,0.850,29,2.397,4.295,5.916,3.017,11,0.149,0.468,0.900,0.137,36,0.000,0.529,2.188,1.167,36,10.956,14.038,20.866,23.880,45,30.207,70.426,94.202,994.051,59,1004.137,1015.721,1023.398,89.588,42,0.241,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,97.0,849,729,1,10199.456268,131.91649,94.311010,75.79660,-56.11989,10199.456268,131.91649,94.311010,75.79660,-56.11989,7328.072022,80.47936,66.136493,16.41142,-64.06794,13010.294810,168.67617,122.643378,104.53181,-64.14436,20190101,12,0,6.885,3.058,3.119,4.763,4.824,0.997,988.640188,14258.691398,15.273344,145.405604,71533.167146,76.623488,729.472508,1105.104448,10520.838118,11.269504
1,2,2019,1,1,1,Darwin,-12.46113,130.84185,47,2.594,3.181,4.828,1.208,49,3.092,7.572,12.332,20.524,31,0.096,0.102,16.152,61.550,45,0.099,0.301,5.411,6.401,71,24.113,30.125,36.678,37.327,50,49.034,74.276,92.975,467.112,69,938.085,970.252,1080.000,7.320,48,0.828,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,97.0,849,729,1,7351.674654,94.72344,67.560018,53.61036,-41.11308,7351.674654,94.72344,67.560018,53.61036,-41.11308,5488.824963,54.83595,49.399828,-5.77482,-49.06113,10164.105029,131.48312,95.892084,82.34557,-49.13755,20190101,12,0,10.753,3.283,3.482,7.674,7.873,0.403,2237.564500,29228.841500,104.624125,717.366625,72066.437552,257.960548,1768.734388,3369.685196,23104.610876,82.702549
2,3,2019,1,1,1,Melbourne,-37.81400,144.96332,17,1.190,1.197,2.200,0.248,123,0.736,12.527,23.432,125.224,36,1.392,1.464,47.652,515.570,111,0.972,2.751,12.230,30.518,148,0.000,0.000,23.226,131.280,138,25.919,69.069,100.000,1292.191,153,880.000,880.000,1018.200,4655.348,125,0.000,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,97.0,849,729,1,10206.768673,134.19778,94.896382,67.73183,-66.46595,10206.768673,134.19778,94.896382,67.73183,-66.46595,8318.849546,82.76065,74.880638,8.34665,-74.41400,12892.900183,170.95746,121.879910,96.46704,-74.49042,20190101,12,0,13.724,2.661,3.948,13.991,15.278,4.215,0.000000,0.000000,0.000000,0.000000,60780.720000,145.528383,907.083177,1854.160000,11557.040000,27.671231
3,4,2019,1,1,1,Newcastle,-32.92953,151.78010,63,4.586,11.044,14.802,24.186,90,1.917,14.141,38.374,304.731,50,2.957,3.230,14.056,25.630,176,14.959,17.471,24.221,20.883,101,16.423,19.819,20.044,3.188,111,73.504,86.920,100.000,85.028,141,1011.703,1016.545,1018.364,10.898,52,0.284,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370,97.0,849,729,1,10418.232918,136.13009,96.694229,74.54861,-61.58148,10418.232918,136.13009,96.694229,74.54861,-61.58148,7890.150242,84.69296,71.163791,15.16343,-69.52953,13192.648423,172.88977,124.549331,103.28382,-69.60595,20190101,12,0,25.185,14.274,28.515,17.371,31.612,20.701,1722.667480,20146.905355,9.968957,211.766015,88358.0914

# 使わないカラムを落とす

In [ ]:
#使わない特徴量を落とす
train.drop(['id', 'City', 'pm25_mid'], axis = 1, inplace=True)
test.drop(['id', 'City'], axis = 1, inplace=True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195941 entries, 0 to 195940
Data columns (total 94 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   year                       195941 non-null  int64  
 1   month                      195941 non-null  int64  
 2   day                        195941 non-null  int64  
 3   Country                    195941 non-null  int64  
 4   lat                        195941 non-null  float64
 5   lon                        195941 non-null  float64
 6   co_cnt                     195941 non-null  int64  
 7   co_min                     195941 non-null  float64
 8   co_mid                     195941 non-null  float64
 9   co_max                     195941 non-null  float64
 10  co_var                     195941 non-null  float64
 11  o3_cnt                     195941 non-null  int64  
 12  o3_min                     195941 non-null  float64
 13  o3_mid                     19

In [ ]:
train.fillna(method='ffill', inplace=True)
test.fillna(method='ffill', inplace=True)

# Stacking

modelの宣言

In [ ]:
import xgboost as xgb

class Model1Xgb:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        xgb_params = {'objective': 'reg:squarederror', #binary:logistic #multi:softprob,
                  'random_state': 10,
                  #'eval_metric': 'rmse'
                  'tree_method': 'gpu_hist',
                  'verbose': 10
                     }
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        evals = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(xgb_params, dtrain, num_boost_round=10000, verbose_eval=100, early_stopping_rounds=50, evals=evals)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

In [ ]:
import lightgbm as lgb

class Model1lgb:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        lgb_params = {'objective': 'rmse',
                  'random_state': 10,
                  'metric': 'rmse',
                  'verbose': 100}
        lgb_train = lgb.Dataset(tr_x, label=tr_y)
        lgb_eval = lgb.Dataset(va_x, label=va_y,reference=lgb_train)
        self.model = lgb.train(lgb_params, lgb_train, valid_sets=lgb_eval, verbose_eval=100, num_boost_round=10000, early_stopping_rounds=50)

    def predict(self, x):
        pred = self.model.predict(x,num_iteration=self.model.best_iteration)
        return pred

In [ ]:
import catboost
class Model1catboost:

    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        #https://catboost.ai/docs/concepts/python-reference_catboostregressor.html
        #catb = catboost.CatBoostClassifier(
        catb = catboost.CatBoostRegressor(
                                    iterations=10000, 
                                    use_best_model=True, 
                                    random_seed=10, 
                                    l2_leaf_reg=3,
                                    depth=6,
                                    loss_function="RMSE",#"CrossEntropy",
                                    #eval_metric = "RMSE", #'AUC',
                                    #classes_coun=3,
                                    verbose=100
                                  )
        self.model = catb.fit(tr_x,tr_y,eval_set=(va_x,va_y),early_stopping_rounds=50)
        print(self.model.score(va_x,va_y))
    def predict(self, x):
        pred = self.model.predict(x)
        return pred

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

class Model1NN:
    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)

        batch_size = 128
        epochs = 10000

        tr_x = self.scaler.transform(tr_x)
        va_x = self.scaler.transform(va_x)

        early_stopping =  EarlyStopping(
                            monitor='val_loss',
                            min_delta=0.0,
                            patience=20,
        )

        model = Sequential()
        model.add(Dense(32, activation='relu', input_shape=(tr_x.shape[1],)))
        model.add(Dropout(0.5))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='mean_squared_error', 
                      #'categorical_crossentropy',#categorical_crossentropy
                      optimizer='adam')

        history = model.fit(tr_x, tr_y,
                            batch_size=batch_size, epochs=epochs,
                            verbose=1,
                            validation_data=(va_x, va_y),
                            callbacks=[early_stopping])
        self.model = model

    def predict(self, x):
        x = self.scaler.transform(x)
        pred = self.model.predict_proba(x).reshape(-1)
        return pred

In [ ]:
from sklearn.svm import LinearSVR

class Model1LinearSVR:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        #params = {"C":np.logspace(0,1,params_cnt),"epsilon":np.logspace(-1,1,params_cnt)}
        self.model = LinearSVR(max_iter=1000,
                               random_state=10,
                               C = 1.0, #損失の係数（正則化係数の逆数）
                               epsilon = 5.0

                              )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [ ]:
from sklearn.svm import SVR

class Model1KernelSVR:

    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        #params = {"kernel":['rbf'],"C":np.logspace(0,1,params_cnt), "epsilon":np.logspace(-1,1,params_cnt)}
        self.model = SVR(kernel='rbf',
                         gamma='auto',
                         max_iter=1000,
                         C = 1.0, #損失の係数（正則化係数の逆数）
                         epsilon = 5.0

                              )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [ ]:
from sklearn.linear_model import Lasso

class Model1Lasso:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        '''
        (1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_1
        #https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
        '''
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = Lasso(
            alpha=1, #L1係数
            fit_intercept=True,
            max_iter=1000,
            random_state=10,
            )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [ ]:
from sklearn.linear_model import Ridge

class Model1Ridge:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        '''
        |y - Xw||^2_2 + alpha * ||w||^2_2
        #https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
        '''
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = Ridge(
            alpha=1, #L2係数
            max_iter=1000,
            random_state=10,
                              )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [ ]:
from sklearn.linear_model import ElasticNet

class Model1ElasticNet:

    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        '''1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2
       ref)  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
        '''

        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = ElasticNet(
            alpha=1, #L1係数
            l1_ratio=0.5,
                              )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [ ]:
from sklearn.ensemble import RandomForestRegressor

class Model1RF:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = RandomForestRegressor(
            max_depth=5,
            n_estimators=100,
            random_state=10,
        )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

class Model1KNN:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        #params = {"kernel":['rbf'],"C":np.logspace(0,1,params_cnt), "epsilon":np.logspace(-1,1,params_cnt)}
        self.model = KNeighborsRegressor(n_neighbors=5,
                                         #weights='uniform'
                                        )

        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [ ]:
from sklearn.linear_model import LinearRegression

class Model2Linear:
    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = LinearRegression()
        self.model.fit(tr_x, tr_y)

    def predict(self, x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=10)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [ ]:
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, train, target, test)

model_1b = Model1lgb()
pred_train_1b, pred_test_1b = predict_cv(model_1b, train, target, test)

#model_1c = Model1NN()
#pred_train_1c, pred_test_1c = predict_cv(model_1c, train, target, test)

model_1d = Model1LinearSVR()
pred_train_1d, pred_test_1d = predict_cv(model_1d, train, target, test)

model_1e = Model1KernelSVR()
pred_train_1e, pred_test_1e = predict_cv(model_1e, train, target, test)

model_1f = Model1catboost()
pred_train_1f, pred_test_1f = predict_cv(model_1f, train, target, test)

model_1g = Model1KNN()
pred_train_1g, pred_test_1g = predict_cv(model_1g, train, target, test)

model_1h = Model1Lasso()
pred_train_1h, pred_test_1h = predict_cv(model_1h, train, target, test)

model_1i = Model1Ridge()
pred_train_1i, pred_test_1i = predict_cv(model_1i, train, target, test)

model_1j = Model1ElasticNet()
pred_train_1j, pred_test_1j = predict_cv(model_1j, train, target, test)

model_1k = Model1RF()
pred_train_1k, pred_test_1k = predict_cv(model_1k, train, target, test)

[0]	train-rmse:52.3615	eval-rmse:51.9523
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:17.0614	eval-rmse:19.6562
[200]	train-rmse:15.1604	eval-rmse:19.5826
[300]	train-rmse:13.702	eval-rmse:19.5606
Stopping. Best iteration:
[302]	train-rmse:13.6721	eval-rmse:19.5577

[0]	train-rmse:52.2933	eval-rmse:52.1699
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:17.0018	eval-rmse:19.7399
[200]	train-rmse:15.0457	eval-rmse:19.6269
[300]	train-rmse:13.631	eval-rmse:19.6247
Stopping. Best iteration:
[313]	train-rmse:13.4506	eval-rmse:19.618

[0]	train-rmse:52.187	eval-rmse:52.4994
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:17.0469	eval-rmse:19.684
[200]	train-

In [ ]:
from sklearn.metrics import mean_absolute_error

#Calculate RMSE
#from sklearn.metrics import mean_square_error
def rmse(y_true,y_pred):
    rmse = np.sqrt(metrics.mean_squared_error(y_true,y_pred))
    print('rmse',rmse)
    return rmse

In [ ]:
#1次元配列に
pred_train_1g = np.ravel(pred_train_1g)
pred_train_1i = np.ravel(pred_train_1i)

pred_test_1g = np.ravel(pred_test_1g)
pred_test_1i = np.ravel(pred_test_1i)

In [ ]:
#モデルの精度の確認
print(f'a LGBM mean_absolute_error: {mean_absolute_error(target, pred_train_1a):.4f}')
print(f'b XGBoostmean_absolute_error: {mean_absolute_error(target, pred_train_1b):.4f}')
#print(f'c MLP mean_absolute_error: {mean_absolute_error(target, pred_train_1c):.4f}')
print(f'd LinearSVR mean_absolute_error: {mean_absolute_error(target, pred_train_1d):.4f}')
print(f'e KernelSVR mean_absolute_error: {mean_absolute_error(target, pred_train_1e):.4f}')
print(f'f Catboost mean_absolute_error: {mean_absolute_error(target, pred_train_1f):.4f}')
print(f'g KNN mean_absolute_error: {mean_absolute_error(target, pred_train_1g):.4f}')
print(f'h Lasso mean_absolute_error: {mean_absolute_error(target, pred_train_1h):.4f}')
print(f'i Ridge mean_absolute_error: {mean_absolute_error(target, pred_train_1i):.4f}')
print(f'j ElasticNet mean_absolute_error: {mean_absolute_error(target, pred_train_1j):.4f}')
print(f'k RandomForest mean_absolute_error: {mean_absolute_error(target, pred_train_1k):.4f}')

a LGBM mean_absolute_error: 14.2161
b XGBoostmean_absolute_error: 13.5585
d LinearSVR mean_absolute_error: 18.4598
e KernelSVR mean_absolute_error: 29.3519
f Catboost mean_absolute_error: 13.4314
g KNN mean_absolute_error: 16.6850
h Lasso mean_absolute_error: 19.5958
i Ridge mean_absolute_error: 18.9813
j ElasticNet mean_absolute_error: 19.6912
k RandomForest mean_absolute_error: 17.9877


In [ ]:
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a,
                          'pred_1b': pred_train_1b,
                          #'pred_1c': pred_train_1c,
                          'pred_1d': pred_train_1d,
                          'pred_1e': pred_train_1e,
                          'pred_1f': pred_train_1f,
                          'pred_1g': pred_train_1g,
                          'pred_1h': pred_train_1h,
                          'pred_1i': pred_train_1i,
                          'pred_1j': pred_train_1j,
                          'pred_1k': pred_train_1k,
                         })


In [ ]:
test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a,
                          'pred_1b': pred_test_1b,
                          #'pred_1c': pred_test_1c,
                          'pred_1d': pred_test_1d,
                          'pred_1e': pred_test_1e,
                          'pred_1f': pred_test_1f,
                          'pred_1g': pred_test_1g,
                          'pred_1h': pred_test_1h,
                          'pred_1i': pred_test_1i,
                          'pred_1j': pred_test_1j,
                          'pred_1k': pred_test_1k
                         })

In [ ]:
model2 = Model2Linear()
pred_train_2, pred_test_2 = predict_cv(model2, train_x_2, target, test_x_2)

#２層のスタッキング後の、精度を確認。
print(f'mean_absolute_error: {mean_absolute_error(target, pred_train_2):.4f}')

mean_absolute_error: 13.2716


In [ ]:
sub_df = pd.DataFrame(pred_test_2, index=submission[0].values)
sub_df.to_csv("93_stacking_decision.csv", header=False)

In [ ]:
sub_df

,0
195942,17.984160
195943,33.209001
195944,30.882117
195945,64.610286
195946,127.101500
...,...
249446,75.831264
249447,92.144801
249448,69.584080
249449,32.425404
